# Data Exploration

This notebook demonstrates how to read a video file, extract frames, and display them using OpenCV and Matplotlib. It also shows how to detect faces in a frame using the Viola-Jones algorithm.

In [ ]:
import cv2

input_video_path = '/Volumes/Patrick/Datasets/VitalCamSet/Proband16/101_natural_lighting/Logitech HD Pro Webcam C920.avi'
video = cv2.VideoCapture(input_video_path)

## Video properties

In [ ]:
# Get the video properties
fps = video.get(cv2.CAP_PROP_FPS)
frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
duration = frame_count / fps

print(f'FPS: {fps}')
print(f'Frame count: {frame_count}')
print(f'Frame: {frame_width}x{frame_height}')
print(f'Duration: {duration:.2f} seconds')

## Display random frames

In [ ]:
import random

# Seed the random number generator for reproducibility
random.seed(42)

frame_inx1 = 20
frame_inx2 = 21
# frame_inx1 = random.randint(0, frame_count)
# frame_inx2 = random.randint(0, frame_count)

video.set(cv2.CAP_PROP_POS_FRAMES, frame_inx1)
_, frame1 = video.read()

video.set(cv2.CAP_PROP_POS_FRAMES, frame_inx2)
_, frame2 = video.read()

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(10, 5))

frame1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB)
axes[0].imshow(frame1)
axes[0].set_title(f'Frame {frame_inx1}')

frame2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB)
axes[1].imshow(frame2)
axes[1].set_title(f'Frame {frame_inx2}')

## Optical flow

In [ ]:
import respiration.utils as utils

device = utils.get_torch_device();

In [ ]:
model_path = '../../data/RAFT/raft-things.pth'

In [ ]:
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--model', help="restore checkpoint")
parser.add_argument('--path', help="dataset for evaluation")
parser.add_argument('--small', action='store_true', help='use small model')
parser.add_argument('--mixed_precision', action='store_true', help='use mixed precision')
parser.add_argument('--alternate_corr', action='store_true', help='use efficent correlation implementation')

args = parser.parse_args([])

In [ ]:
import torch
import raft

model = torch.nn.DataParallel(raft.RAFT(args))
model.load_state_dict(torch.load(model_path, map_location=device))

model = model.module
model.to(device)
model.eval()

with torch.no_grad():
    image1 = torch.tensor(frame1, device=device, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0)
    image2 = torch.tensor(frame2, device=device, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0)

    flow_low, flow_up = model(image1, image2, iters=20, test_mode=True)

In [ ]:
flow_low.shape, flow_up.shape

In [ ]:
import matplotlib.pyplot as plt
import raft.utils as raft_utils

# Display flow_low, flow_up
_, axes = plt.subplots(1, 2, figsize=(20, 8))

axes[0].imshow(raft_utils.flow_to_image(flow_low[0].permute(1, 2, 0).cpu().numpy()))
axes[0].set_title('Flow low')

axes[1].imshow(raft_utils.flow_to_image(flow_up[0].permute(1, 2, 0).cpu().numpy()))
axes[1].set_title('Flow up')

## Create a motion video

In [ ]:
import respiration.dataset as dt

dataset = dt.from_default()

subject = 'Proband16'
setting = '101_natural_lighting'

frames, _ = dataset.get_video_rgb(subject, setting, num_frames=1000, show_progress=True)

In [ ]:
import tqdm.auto as tqdm

of_frames = []

for idx in tqdm.tqdm(range(1, len(frames))):
    frame1 = frames[idx - 1]
    frame2 = frames[idx]

    with torch.no_grad():
        image1 = torch.tensor(frame1, device=device, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0)
        image2 = torch.tensor(frame2, device=device, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0)

        flow_low, flow_up = model(image1, image2, iters=20, test_mode=True)

    flow_up = flow_up[0].permute(1, 2, 0).cpu().numpy()
    flow_up = cv2.resize(flow_up, (frame1.shape[1], frame1.shape[0]))

    of_frames.append(raft.flow_to_image(flow_up))

In [ ]:
import cv2
import numpy as np

# Write the video to a file in avi format
out = cv2.VideoWriter('motion.avi', cv2.VideoWriter_fourcc(*'DIVX'), 30, (frame1.shape[1], frame1.shape[0]))

for frame in of_frames:
    out.write((frame * 255).astype(np.uint8))

out.release()